In [7]:
import urllib.request
from netCDF4 import Dataset
import numpy as np
from numpy import asarray
import PIL

import xarray as xr
import rioxarray as rio
from PIL import Image
PIL.Image.MAX_IMAGE_PIXELS = 933120000
import os
import shutil

from joblib import Parallel, delayed
import time
import multiprocessing as mp

In [8]:
def get_image_from_tile(BBOX,date_string):
    '''
    Downloads the image from the WMS tile and returns the path of downloaded image.
    Input parameters:
    BBOX: Bounding box coordinates(ln_w, lt_s, ln_e, lt_n)
    date_string: "yyyy_dd_mm_hh" hh is optional

    '''
    floods_url =  "https://bhuvan-gp1.nrsc.gov.in/bhuvan/gwc/service/wms?LAYERS=flood%3Aas_"+date_string+"&TRANSPARENT=TRUE&SERVICE=WMS&VERSION=1.1.1&REQUEST=GetMap&STYLES=&FORMAT=image%2Fpng&SRS=EPSG%3A4326&BBOX="+BBOX+"&WIDTH=256&HEIGHT=256"
    #print(floods_url)
    urllib.request.urlretrieve(floods_url, r"Tiles/"+date_string+"xx"+BBOX+".image")    
    return None #r"Tiles/"+date_string+"xx"+BBOX+".image"

In [9]:
#FUNCTION NOT USED
def get_grayscale_array(image_path):
    '''
    Converts the RGBA image into grayscale and returns the array.
    Input parameters
    image_path: file path of the image.
    
    Returns the grayscale image and its path.
    '''
    image = Image.open(image_path)
    grayscale_image = image.convert('L')
    return grayscale_image, asarray(grayscale_image)

In [10]:
def merge_images(file1, file2,horizontal):
    """Merge two images into one
    Input parameters
    file1: path to first image file
    file2: path to second image file
    horizontal: True if the images are to be stitched horizontally
    
    returns the merged Image object
    """
    if type(file1) ==PIL.Image.Image:
        image1 = file1
    else:
        image1 = Image.open(file1)
        #Removing BHUVAN watermark
        try:
            image1_ar = np.asarray(image1)
            #image1_ar[:,:,3][(image1_ar[:,:,3]<255.0)] = 0
            image1_ar[:,:,:3][(image1_ar[:,:,0]==image1_ar[:,:,1])&((image1_ar[:,:,1]==image1_ar[:,:,2]))] = 255.0
            image1 = Image.fromarray(image1_ar)
        except:
            pass
    
    image2 = Image.open(file2)
    #Removing BHUVAN watermark
    try:
        image2_ar = np.asarray(image2)
        #image2_ar[:,:,3][(image2_ar[:,:,3]<255.0)] = 0
        image2_ar[:,:,:3][(image2_ar[:,:,0]==image2_ar[:,:,1])&((image2_ar[:,:,1]==image2_ar[:,:,2]))] = 255.0
        image2 = Image.fromarray(image2_ar)
    except:
        pass

    (width1, height1) = image1.size
    (width2, height2) = image2.size

    if horizontal==True:
        result_width = width1 + width2
        result_height = max(height1, height2)
        result = Image.new('L', (result_width, result_height))
        result.paste(im=image1, box=(0, 0))
        result.paste(im=image2, box=(width1, 0))
    else:
        result_height = height1 + height2
        result_width = max(width1,width2)
        result = Image.new('L', (result_width, result_height))
        result.paste(im=image1, box=(0, 0))
        result.paste(im=image2, box=(0, height1))

    #result.save('result.image')

    return result

In [11]:
def create_nc_from_images(lt_s,lt_n,ln_w,ln_e,image_path,image_name):
    '''
    Creates NetCDF4 files from the grayscale image using the BBOX coordinates.
    
    Input parameters:
    BBOX coordindates (lt_s, lt_n, ln_w, ln_e)
    image_path: filepath of the image to be converted.
    image_name: output file name
    
    Returns the filepath of the NetCDF4 file created.
    '''
    image = Image.open(image_path)
    grayscale_array = np.asarray(image)
    
    lt_array = np.linspace(lt_n,lt_s,grayscale_array.shape[0])
    ln_array = np.linspace(ln_w,ln_e,grayscale_array.shape[1])
    
    my_file = Dataset(r"NCs/"+image_name+'.nc','w',format='NETCDF4')
    lat_dim = my_file.createDimension('lat',grayscale_array.shape[0])
    lon_dim = my_file.createDimension('lon',grayscale_array.shape[1])
    time_dim = my_file.createDimension('time',None)
    
    latitudes = my_file.createVariable("lat", 'f4', ('lat',))
    longitudes = my_file.createVariable("lon", 'f4', ('lon',))
    time = my_file.createVariable('time', np.float32, ('time',))
    
    new_nc_variable = my_file.createVariable("Inundation", np.float32, ('time','lat','lon'))
    latitudes[:] = lt_array
    longitudes[:] = ln_array
    new_nc_variable[0,:,:] = grayscale_array
    
    my_file.close()
    
    return r"NCs/"+image_name+'.nc'

In [12]:
def create_tiffs_from_ncs(nc_path,image_name):
    '''
    Creates GeoTIFF files from the NetCDF4 files.
    
    Input parameters:
    nc_path: filepath of the NetCDF4 file.
    image_name: Output name of the GeoTIFF
    '''
    
    tiff_file = xr.open_dataset(nc_path)
    var = tiff_file['Inundation']
    var = var.rio.set_spatial_dims('lon','lat')
    var.rio.set_crs("epsg:4326")
    var.rio.to_raster(r"tiffs/"+image_name+r".tif")
    tiff_file.close()
    

In [13]:

date_string = "2020_28_05"
delta = 0.0439453125

starting_point_lat = 23.994140625
lt_s = starting_point_lat

starting_point_lon=90 -7*delta
ln_w = starting_point_lon
BBOXs=[]

#Scraping the tile longitudinally first and then latitudinally.
tic = time.perf_counter()

no_images_vertically=0
while lt_s <= 28.125:
    lt_n = lt_s+delta
        
    no_images_horizontally = 0
    while ln_w <= 96:
        ln_e = ln_w+delta   
        BBOX="{},{},{},{}".format(ln_w,lt_s,ln_e,lt_n)
        ## Download the tile image
        #get_image_from_tile(BBOX,date_string)
        BBOXs.append(BBOX)
        no_images_horizontally = no_images_horizontally+1
           
        ln_w = ln_e
    

    if no_images_vertically==0:
            print("Number of images horizontally: ",no_images_horizontally)
        
    
    lt_s = lt_n
    ln_w = starting_point_lon
    no_images_vertically=no_images_vertically+1

print("Number of vertical images: ",no_images_vertically)
Parallel(n_jobs=mp.cpu_count())(delayed(get_image_from_tile)(BBOX,date_string) for BBOX in BBOXs)

## All tile images are downloaded -- these have to be stitched.
#144 tiles horizontally.
toc = time.perf_counter()
print("Time Taken: {} seconds".format(toc-tic))

Number of images horizontally:  144
Number of vertical images:  95
Time Taken: 1161.0435613 seconds


In [14]:
import glob
extension = 'image'
        #os.chdir(path)
result = glob.glob('Tiles/*.{}'.format(extension))

lats = []
lons = []
for file in result:
    lats.append(float(file.split(',')[-1].split('.image')[0]))
    lons.append(float(file.split(',')[-2]))

In [15]:
tic = time.perf_counter()
c = 0
count=1
#We will first stitch images vertically (latitudinally) and then stitch them horizontally(longitudinally)
#There are 95 images per a vertical stretch
while c+no_images_vertically<=len(result):
    print(count)
    vert_images = result[c:c+no_images_vertically]
    vert_images.reverse()
    print(len(vert_images))
    merged_image = vert_images[0]
    for image in vert_images[1:]:
        merged_image = merge_images(merged_image, image,horizontal=False)
    c=c+no_images_vertically
    
    merged_image.save(r'vert/'+str(count)+'.png')
    count=count+1


#shutil.rmtree('Tiles')
#os.makedirs('Tiles')

toc = time.perf_counter()
print("Time Taken: {} seconds".format(toc-tic))

1
95
2
95
3
95
4
95
5
95
6
95
7
95
8
95
9
95
10
95
11
95
12
95
13
95
14
95
15
95
16
95
17
95
18
95
19
95
20
95
21
95
22
95
23
95
24
95
25
95
26
95
27
95
28
95
29
95
30
95
31
95
32
95
33
95
34
95
35
95
36
95
37
95
38
95
39
95
40
95
41
95
42
95
43
95
44
95
45
95
46
95
47
95
48
95
49
95
50
95
51
95
52
95
53
95
54
95
55
95
56
95
57
95
58
95
59
95
60
95
61
95
62
95
63
95
64
95
65
95
66
95
67
95
68
95
69
95
70
95
71
95
72
95
73
95
74
95
75
95
76
95
77
95
78
95
79
95
80
95
81
95
82
95
83
95
84
95
85
95
86
95
87
95
88
95
89
95
90
95
91
95
92
95
93
95
94
95
95
95
96
95
97
95
98
95
99
95
100
95
101
95
102
95
103
95
104
95
105
95
106
95
107
95
108
95
109
95
110
95
111
95
112
95
113
95
114
95
115
95
116
95
117
95
118
95
119
95
120
95
121
95
122
95
123
95
124
95
125
95
126
95
127
95
128
95
129
95
130
95
131
95
132
95
133
95
134
95
135
95
136
95
137
95
138
95
139
95
140
95
141
95
142
95
143
95
144
95
Time Taken: 76.52371049999988 seconds


In [16]:
#Stitch images horizontally
import glob
import natsort
tic = time.perf_counter()

extension = 'png'
vert_imgs = glob.glob('vert/*.{}'.format(extension))
vert_imgs = natsort.natsorted(vert_imgs,reverse=False)

merged_image = vert_imgs[0]
print(vert_imgs[0])
for image in vert_imgs[1:]:
    print(image)
    merged_image = merge_images(merged_image, image,horizontal=True)


toc = time.perf_counter()
print("Time Taken: {} seconds".format(toc-tic))

vert\1.png
vert\2.png
vert\3.png
vert\4.png
vert\5.png
vert\6.png
vert\7.png
vert\8.png
vert\9.png
vert\10.png
vert\11.png
vert\12.png
vert\13.png
vert\14.png
vert\15.png
vert\16.png
vert\17.png
vert\18.png
vert\19.png
vert\20.png
vert\21.png
vert\22.png
vert\23.png
vert\24.png
vert\25.png
vert\26.png
vert\27.png
vert\28.png
vert\29.png
vert\30.png
vert\31.png
vert\32.png
vert\33.png
vert\34.png
vert\35.png
vert\36.png
vert\37.png
vert\38.png
vert\39.png
vert\40.png
vert\41.png
vert\42.png
vert\43.png
vert\44.png
vert\45.png
vert\46.png
vert\47.png
vert\48.png
vert\49.png
vert\50.png
vert\51.png
vert\52.png
vert\53.png
vert\54.png
vert\55.png
vert\56.png
vert\57.png
vert\58.png
vert\59.png
vert\60.png
vert\61.png
vert\62.png
vert\63.png
vert\64.png
vert\65.png
vert\66.png
vert\67.png
vert\68.png
vert\69.png
vert\70.png
vert\71.png
vert\72.png
vert\73.png
vert\74.png
vert\75.png
vert\76.png
vert\77.png
vert\78.png
vert\79.png
vert\80.png
vert\81.png
vert\82.png
vert\83.png
vert\84.png
v

In [17]:
tic = time.perf_counter()
merged_image_ar = np.asarray(merged_image)
merged_image_ar[:,:][(merged_image_ar[:,:]<255)] = 179
merged_image = Image.fromarray(merged_image_ar)
merged_image.save('offset2.png')
#shutil.rmtree('vert')
#os.makedirs('vert')
toc = time.perf_counter()
print("Time Taken: {} seconds".format(toc-tic))

Time Taken: 12.649902399999974 seconds


In [18]:
tic = time.perf_counter()
nc_path = create_nc_from_images(starting_point_lat,max(lats),starting_point_lon,max(lons),'offset2.png','offset2')
toc = time.perf_counter()
print("Time Taken: {} seconds".format(toc-tic))

Time Taken: 39.35371370000007 seconds


In [19]:
tic = time.perf_counter()
create_tiffs_from_ncs(nc_path,'offset2')
toc = time.perf_counter()
print("Time Taken: {} seconds".format(toc-tic))

Time Taken: 75.93279619999998 seconds
